In [1]:
# import sys
# from PySide2.QtWidgets import QApplication, QMainWindow, QMessageBox
# from BLDC_UI import Ui_MainWindow  # Replace 'your_ui_file' with the actual filename if needed

# class MainWindow(QMainWindow):
#     def __init__(self):
#         super(MainWindow, self).__init__()
#         self.ui = Ui_MainWindow()
#         self.ui.setupUi(self)
#         self.ui.comboBox.addItem("Hello")
#         self.ui.ParallelPlot.clicked.connect(self.on_button_clicked)
#         self.ui.Execution.clicked.connect(self.on_Execution_clicked)
        
#     def on_Execution_clicked(self):
#         print(self.ui.R_MagnetRadialLength_unit.text())

#     def on_button_clicked(self):
#         self.ui.ParallelPlot.setText("Button Pressed!")  # Change label text when button is pressed
#         msg = QMessageBox()
#         msg.setWindowTitle("Title")
#         x = msg.exec_()  


# if __name__ == "__main__":
#     app = QApplication(sys.argv)
#     mainWin = MainWindow()
#     mainWin.show()
#     app.exec_()

In [ ]:
import sys
from PySide2.QtWidgets import QApplication, QMainWindow
from BLDC_UI import Ui_MainWindow  # Import the UI class generated by Qt Designer
from pint import UnitRegistry  # Import Pint for unit conversion
from pymoo.core.variable import Real, Integer, Choice, Binary  # Import Pymoo variable types
from UI_MotorOptimizer import MotorOpt
from UI_MotorModel import MotorCalcs_Vectorized
import matlab.engine 
eng = matlab.engine.start_matlab()

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)  # Set up the UI

        # Create a UnitRegistry instance for unit conversion
        self.ureg = UnitRegistry()

        # Connect the Execution button to the slot
        self.ui.Execution.clicked.connect(self.optimizer)

    def optimizer(self):
        variables = {}
        parameters = {} 
        OptimizationConf = {}

        # Collect user inputs from the UI elements
        variables['I_PolePair'] = self.ui.I_PolePair_1.value()
        variables['R_MagnetRadialLength'] = self.ui.R_MagnetRadialLength_1.value()
        variables['R_MagnetOutR'] = self.ui.R_MagnetOutR_1.value()
        variables['R_MagnetBackIronInnerR'] = self.ui.R_MagnetBackIronInnerR_1.value()
        variables['R_MagneticLossCoef1'] = self.ui.R_MagneticLossCoef1_1.value()
        variables['R_MagneticLossCoef2'] = self.ui.R_MagneticLossCoef2_1.value()
        variables['R_MagneticLossCoef3'] = self.ui.R_MagneticLossCoef3_1.value()
        variables['I_NumElecPhase'] = self.ui.I_NumElecPhase_1.value()
        variables['R_PhaseCurrentAmp'] = self.ui.R_PhaseCurrentAmp_1.value()
        variables['R_WireCopperR'] = self.ui.R_WireCopperR_1.value()
        variables['R_InsulThick'] = self.ui.R_InsulThick_1.value()
        variables['R_FillFactor'] = self.ui.R_FillFactor_1.value()
        variables['R_LaminatToothWedgeThickness'] = self.ui.R_LaminatToothWedgeThickness_1.value()
        variables['R_LaminatR'] = self.ui.R_LaminatR_1.value()
        variables['R_LamSlotLinerThick'] = self.ui.R_LamSlotLinerThick_1.value()
        variables['R_LoadInertia'] = self.ui.R_LoadInertia_1.value()
        variables['R_GearRatio'] = self.ui.R_GearRatio_1.value()
        variables['R_MtrLength'] = self.ui.R_MtrLength_1.value()
        variables['R_FluidSpecificGrav'] = self.ui.R_FluidSpecificGrav_1.value()
        variables['R_FluidKinVisc'] = self.ui.R_FluidKinVisc_1.value()
        variables['R_AirgapRadialLength'] = self.ui.R_AirgapRadialLength_1.value()
        variables['R_VacMagneticPerm'] = self.ui.R_VacMagneticPerm_1.value()
        
        OptimizationConf['SpeedReq'] = self.ui.SpeedReq.text()
        OptimizationConf['MinTor'] = self.ui.MinTor.text()
        OptimizationConf['MinEff_'] = self.ui.MinEff_.value()
        OptimizationConf['MaxWeight'] = self.ui.MaxWeight.value()
        OptimizationConf['VDC'] = self.ui.VDC.value()
        OptimizationConf['TorqueOpt'] = self.ui.TorqueOpt.isChecked()
        OptimizationConf['EfficinecyOpt_min'] = self.ui.EfficinecyOpt_min.isChecked()
        OptimizationConf['EfficinecyOpt_max'] = self.ui.EfficinecyOpt_max.isChecked()
        OptimizationConf['WeightOpt'] = self.ui.WeightOpt.isChecked()
        OptimizationConf['MagWeightOpt'] = self.ui.MagWeightOpt.isChecked()
        OptimizationConf['VoltageOpt'] = self.ui.VoltageOpt.isChecked()
        OptimizationConf['MtrLengthOpt'] = self.ui.MtrLengthOpt.isChecked()
        OptimizationConf['MtrRadiusOpt'] = self.ui.MtrRadiusOpt.isChecked()
        OptimizationConf['npop'] = self.ui.npop.value()
        OptimizationConf['ngens'] = self.ui.ngens.value()
        OptimizationConf['nprocesses'] = self.ui.nprocesses.value()
        OptimizationConf['DisMut'] = self.ui.DisMut.value()
        OptimizationConf['ProbMut'] = self.ui.ProbMut.value()
        

        #print(OptimizationConf.get("SpeedReq"))
        self.update_magnet_br()
        self.update_max_b_magnet_iron()  # Update MaxBMagnetIron_T based on current selection
        self.update_present_b_lamination()  # Update PresentBLaminationBackIron_T based on current selection

        for key, value in variables.items():
            if isinstance(value, int):
                parameters[key] = Integer(value=value, bounds=(getattr(self.ui, key + '_1').value(), getattr(self.ui, key + '_2').value()))
            elif isinstance(value, float): 

                # unit = getattr(self.ui, key + '_unit').value()
                # print(unit)
                parameters[key] = Real(value=value, bounds=(getattr(self.ui, key + '_1').value(), getattr(self.ui, key + '_2').value()))

        res = MotorOpt(MotorCalcs_Vectorized, parameters, self.MagnetBr_T, self.MaxBMagnetIron_T, self.PresentBLaminationBackIron_T \
                       ,eng, Real, Integer, Choice, Binary,OptimizationConf) 
    

    def update_magnet_br(self):
        # Define the dictionary mapping materials to their float values
        magnet_br_values = {
            "Recoma 18": 0.87,
            "Recoma 20": 0.90,
            "Recoma 22": 0.94,
            "Recoma 25": 1.00,
            "Recoma 24HE": 1.02,
            "Recoma 26": 1.04,
            "Recoma 26HE": 1.07,
            "Recoma 28": 1.10,
            "Recoma 28HE": 1.10,
            "Recoma 30": 1.12,
            "Recoma 30HE": 1.12,
            "Recoma 30S": 1.12,
            "Recoma 32": 1.15,
            "Recoma 32S": 1.15,
            "Recoma 33E": 1.16,
            "Recoma 35E": 1.19,
        }
    
        # Get the selected material from the QComboBox
        selected_material = self.ui.Magnet_Material.currentText()
        self.MagnetBr_T = magnet_br_values.get(selected_material, 1.1)

    def update_max_b_magnet_iron(self):
        """Update the MaxBMagnetIron_T based on the selected iron material."""
        selected_option = self.ui.Iron_Material.currentText()
        iron_br_values = {
            "430 Stainless Steel": 1.3,
            "1215 Steel": 1.5,
            "1018 Steel": 1.4,
            "1008 Steel": 1.4,
            "1010 Steel": 1.4,
            "Hiperco 50": 2.0,
            "M19": 1.75,
        }
        self.MaxBMagnetIron_T = iron_br_values.get(selected_option, 1.5)  # Default to 1.5 if not found

    def update_present_b_lamination(self):
        """Update the PresentBLaminationBackIron_T based on the selected lamination material."""
        selected_option = self.ui.Lamination_Material.currentText()
        lamination_br_values = {
            "M19": 1.75,
            "Hiperco 50": 2.3,
        }
        self.PresentBLaminationBackIron_T = lamination_br_values.get(selected_option, 1.75)  # Default to 1.75 if not found


    def get_unit(self, variable_name):
        """Return the unit corresponding to the variable name."""
        # Mapping of variable names to units
        unit_map = {
            'R_MagnetRadialLength': 'meter',  # Example: replace with actual units
            'R_MagnetOutR': 'meter',
            'R_MagnetBackIronInnerR': 'meter',
            'R_MagneticLossCoef1': '1',  # Dimensionless
            'R_MagneticLossCoef2': '1',
            'R_MagneticLossCoef3': '1',
            'R_PhaseCurrentAmp': 'ampere',
            'R_WireCopperR': 'meter',
            'R_InsulThick': 'meter',
            'R_FillFactor': '1',  # Dimensionless
            'R_LaminatToothWedgeThickness': 'meter',
            'R_LaminatR': 'meter',
            'R_LoadInertia': 'kg*m^2',
            'R_GearRatio': '1',  # Dimensionless
            'R_MtrLength': 'meter',
            'R_FluidSpecificGrav': '1',  # Dimensionless
            'R_FluidKinVisc': 'cSt',
            'R_AirgapRadialLength': 'meter',
            'R_VacMagneticPerm': 'H/m',
        }
        return unit_map.get(variable_name, 'meter')  # Default to 'meter' if not found

if __name__ == "__main__":
    app = QApplication(sys.argv)
    mainWin = MainWindow()
    mainWin.show()
    sys.exit(app.exec_())

4
6
n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |       10 |  4.700094E+04 |  4.705347E+04 |             - |             -
     2 |       20 |  4.699705E+04 |  4.702497E+04 |             - |             -
     3 |       30 |  4.699694E+04 |  4.700906E+04 |             - |             -
     4 |       40 |  4.699623E+04 |  4.700210E+04 |             - |             -
     5 |       50 |  4.698146E+04 |  4.699580E+04 |             - |             -
     6 |       60 |  4.698146E+04 |  4.699247E+04 |             - |             -
     7 |       70 |  4.698056E+04 |  4.698703E+04 |             - |             -
     8 |       80 |  4.698056E+04 |  4.698143E+04 |             - |             -
     9 |       90 |  4.698056E+04 |  4.698090E+04 |             - |             -
    10 |      100 |  4.697947E+04 |  4.698068E+04 |             - |             -
There is no solution. Consider relaxing the constraints
